In [1]:
using Plots
using LinearAlgebra

In [60]:
struct stat
    state::Array{Float64,1}
    function stat(zero, up, down, double)
        norm = sqrt(zero * zero'+ up * up'+ down * down'+ double * double')
        new([zero,up,down,double] / norm)
    end
    function stat(state::Array)
       new(state) 
    end
end

In [62]:
struct sysstat
    state::Array{stat,2}
    function sysstat(state::Array{Int64,2})
        for n in state 
            @assert(1<=n<=4,"There are only 4 states per site. You have to choose your state within that range")
        end
        s = []
        push!(s, stat(1,0,0,0))
        push!(s, stat(0,1,0,0))
        push!(s, stat(0,0,1,0))
        push!(s, stat(0,0,0,1))
        println(s)
        actual_state = Array{stat,2}(undef, size(state)[1], size(state)[2])
        for i in 1:size(state)[1]
            for j in 1:size(state)[2]
               actual_state[i, j] = s[state[i, j]]
            end
        end
        new(actual_state)
    end
    function sysstat(state::Array{stat,2})
        new(state)
    end
end

In [49]:
import Base.*

function *(a::Array,b::stat)
    @assert(size(a)[1] == length(b.state), "The operator's dimension should match the state (4 by 4)")
    @assert(size(a)[2] == length(b.state), "The operator's dimension should match the state (4 by 4)")
    c = a * b.state
    return stat(c)
end

function *(a::stat,b::stat)
    c = a.state' * b.state
    return c
end

* (generic function with 407 methods)

In [76]:
function *(a::Array, b::sysstat)
    @assert(size(a) == size(b.state), "The operator's dimension should match the systemstate's.")
    c = Array{stat,2}(undef, size(b.state)[1], size(b.state)[2])
    for i in 1:size(a)[1]
        for j in 1:size(a)[2]
            c[i, j] = a[i, j] * b.state[i, j]
        end
    end
    return sysstat(c)
end

function *(a::sysstat, b::sysstat)
    @assert(size(a.state) == size(b.state), "The size of system doesn't match") 
    c = 1
    for i in 1:size(a.state)[1]
        for j in 1:size(a.state)[2]
            c = c * (a.state[i, j] * b.state[i, j])
        end
    end
    return c
end

function *(a::Union{Float64,Int64}, b::stat)
    c = a * b.state
    return stat(c)
end

function *(a::Union{Float64,Int64}, b::sysstat)
    c = Array{stat,2}(undef, size(b.state)[1], size(b.state)[2])
    for i in 1:size(b.state)[1]
        for j in 1:size(b.state)[2]
            c[i, j] = a * b.state[i, j]
        end
    end
    sysstat(c)
end

* (generic function with 414 methods)

In [63]:
fill([0 0 0 0;1 0 0 0;0 0 0 0;0 0 1 0],(2,2)) * sysstat([1 2;3 4])

Any[stat([1.0, 0.0, 0.0, 0.0]), stat([0.0, 1.0, 0.0, 0.0]), stat([0.0, 0.0, 1.0, 0.0]), stat([0.0, 0.0, 0.0, 1.0])]


sysstat(stat[stat([0.0, 1.0, 0.0, 0.0]) stat([0.0, 0.0, 0.0, 0.0]); stat([0.0, 0.0, 0.0, 1.0]) stat([0.0, 0.0, 0.0, 0.0])])

In [77]:
10 *sysstat([1 2;3 4])

Any[stat([1.0, 0.0, 0.0, 0.0]), stat([0.0, 1.0, 0.0, 0.0]), stat([0.0, 0.0, 1.0, 0.0]), stat([0.0, 0.0, 0.0, 1.0])]


sysstat(stat[stat([10.0, 0.0, 0.0, 0.0]) stat([0.0, 10.0, 0.0, 0.0]); stat([0.0, 0.0, 10.0, 0.0]) stat([0.0, 0.0, 0.0, 10.0])])

In [68]:
sysstat([1 2;3 4]) * sysstat([1 2;3 4])

Any[stat([1.0, 0.0, 0.0, 0.0]), stat([0.0, 1.0, 0.0, 0.0]), stat([0.0, 0.0, 1.0, 0.0]), stat([0.0, 0.0, 0.0, 1.0])]
Any[stat([1.0, 0.0, 0.0, 0.0]), stat([0.0, 1.0, 0.0, 0.0]), stat([0.0, 0.0, 1.0, 0.0]), stat([0.0, 0.0, 0.0, 1.0])]


1.0

In [72]:
import Base.+

function +(a::stat,b::stat)
    c = a.state + b.state
    return stat(c)
end

function +(a::sysstat, b::sysstat)
    c = Array{stat,2}(undef, size(b.state)[1], size(b.state)[2])
    for i in 1:size(a.state)[1]
        for j in 1:size(a.state)[2]
            c[i ,j] = a.state[i, j] + b.state[i, j]
        end
    end
    return sysstat(c)
end

+ (generic function with 190 methods)

In [73]:
sysstat([1 2;3 4]) + sysstat([1 2;3 4])

Any[stat([1.0, 0.0, 0.0, 0.0]), stat([0.0, 1.0, 0.0, 0.0]), stat([0.0, 0.0, 1.0, 0.0]), stat([0.0, 0.0, 0.0, 1.0])]
Any[stat([1.0, 0.0, 0.0, 0.0]), stat([0.0, 1.0, 0.0, 0.0]), stat([0.0, 0.0, 1.0, 0.0]), stat([0.0, 0.0, 0.0, 1.0])]


sysstat(stat[stat([2.0, 0.0, 0.0, 0.0]) stat([0.0, 2.0, 0.0, 0.0]); stat([0.0, 0.0, 2.0, 0.0]) stat([0.0, 0.0, 0.0, 2.0])])

In [71]:
I = Matrix{Float64}(I,4,4)

4×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [14]:
size(A)

(3, 2)

In [7]:
[0 0 0 0;1 0 0 0;0 0 0 0;0 0 1 0]* 

4×4 Array{Int64,2}:
 0  0  0  0
 0  1  0  0
 0  0  0  0
 0  0  0  1

In [8]:
[0 1 0 0;0 0 0 0;0 0 0 1;0 0 0 0]*[0 0 0 0;1 0 0 0;0 0 0 0;0 0 1 0]

4×4 Array{Int64,2}:
 1  0  0  0
 0  0  0  0
 0  0  1  0
 0  0  0  0

In [12]:
typeof(1.1+2.2im)


Complex{Float64}

In [13]:
function haha(x::Complex{Float64})
  println(x)  
end
haha(2.0)

MethodError: MethodError: no method matching haha(::Float64)
Closest candidates are:
  haha(!Matched::Complex{Float64}) at In[13]:2

In [79]:
diagm([1 2;3 4], [1 2;3 4])

MethodError: MethodError: no method matching diagm(::Array{Int64,2}, ::Array{Int64,2})